In [19]:
import numpy as np
import pandas as pd
import chess
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm  # For Jupyter Notebook
import torch.nn.functional as F
import torch.nn as nn
# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'gpu'
print(f"Using device: {device}")

Using device: cuda


In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ResidualBlock(nn.Module):
    def __init__(self, channels, dropout=0.1):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(channels)
        self.dropout = nn.Dropout2d(dropout)

    def forward(self, x):
        residual = x
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.dropout(out)  # Add dropout between convs
        out = self.bn2(self.conv2(out))
        return F.leaky_relu(out + residual)

class EvalResNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(20, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU()
        )

        self.resblock1 = nn.Sequential(
            ResidualBlock(64),
            ResidualBlock(64)
        )

        self.downsample = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU()
        )

        self.resblock2 = nn.Sequential(
            ResidualBlock(128),
            ResidualBlock(128)
        )

        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))  # [B, 128, 1, 1]

        self.fc = nn.Sequential(
            nn.Flatten(),  # [B, 128]
            nn.Linear(128, 128),
            nn.LeakyReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),
            nn.Tanh()  # Output ∈ [-1, 1]
        )

    def forward(self, x):
        x = self.initial(x)
        x = self.resblock1(x)
        x = self.downsample(x)
        x = self.resblock2(x)
        x = self.global_pool(x)
        x = self.fc(x)
        return x.squeeze(1)


In [21]:
import chess
import torch
import numpy as np

# Load model
model = EvalResNet()
model.load_state_dict(torch.load("eval_only_bestMSE.pth", map_location=device))  # or "cuda" if needed
model.eval()
model.to(device)
device = next(model.parameters()).device

@torch.no_grad()
def board_to_tensor(board: chess.Board):
    tensor = torch.zeros((20, 8, 8), dtype=torch.float32, device=device)
    
    # Optimized piece mapping
    piece_map = {
        'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
        'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
    }
    
    # Vectorized piece placement
    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece:
            rank, file = 7 - square // 8, square % 8
            tensor[piece_map[piece.symbol()], rank, file] = 1
    
    # Metadata channels
    tensor[12].fill_(float(board.turn))
    tensor[13].fill_(float(board.has_kingside_castling_rights(chess.WHITE)))
    tensor[14].fill_(float(board.has_queenside_castling_rights(chess.WHITE)))
    tensor[15].fill_(float(board.has_kingside_castling_rights(chess.BLACK)))
    tensor[16].fill_(float(board.has_queenside_castling_rights(chess.BLACK)))
    tensor[17].fill_(float(board.has_legal_en_passant()))
    tensor[18].fill_(board.halfmove_clock / 50.0)
    tensor[19].fill_(board.fullmove_number / 100.0)
    
    return tensor.unsqueeze(0)


In [22]:
class TranspositionTable:
    def __init__(self, max_size=1000000):
        self.table = {}
        self.max_size = max_size
    
    def get(self, key, depth):
        if key in self.table:
            stored_depth, value = self.table[key]
            if stored_depth >= depth:
                return value
        return None
    
    def store(self, key, depth, value):
        if len(self.table) >= self.max_size:
            # Simple LRU: remove 10% of entries
            items_to_remove = len(self.table) // 10
            keys_to_remove = list(self.table.keys())[:items_to_remove]
            for k in keys_to_remove:
                del self.table[k]
        
        if key not in self.table or self.table[key][0] < depth:
            self.table[key] = (depth, value)

transposition_table = TranspositionTable()

In [23]:
import torch
from torch.utils.data import Dataset
import chess

class EvalOnlyChessDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe.values
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        fen, evaluation = self.data[idx]  # evaluation is Stockfish eval (e.g. +1.23, -3.50, +M3 etc.)

        board_tensor = torch.zeros((20, 8, 8), dtype=torch.float32)
        board = chess.Board(fen)

        piece_map = {
            'P': 0, 'N': 1, 'B': 2, 'R': 3, 'Q': 4, 'K': 5,
            'p': 6, 'n': 7, 'b': 8, 'r': 9, 'q': 10, 'k': 11
        }

        for square in chess.SQUARES:
            piece = board.piece_at(square)
            if piece:
                rank, file = 7 - square // 8, square % 8
                board_tensor[piece_map[piece.symbol()], rank, file] = 1

        # Metadata (channels 12-19)
        board_tensor[12] = int(board.turn)
        board_tensor[13] = int(board.has_kingside_castling_rights(chess.WHITE))
        board_tensor[14] = int(board.has_queenside_castling_rights(chess.WHITE))
        board_tensor[15] = int(board.has_kingside_castling_rights(chess.BLACK))
        board_tensor[16] = int(board.has_queenside_castling_rights(chess.BLACK))
        board_tensor[17] = int(board.has_legal_en_passant())
        board_tensor[18] = board.halfmove_clock / 50.0
        board_tensor[19] = board.fullmove_number / 100.0

        # Normalize evaluation
        eval_str = str(evaluation)
        if "M" in eval_str or "#":
            # Treat mate as ±100
            eval_value = 100.0 if "+" in eval_str else -100.0
        else:
            eval_value = float(eval_str)

        eval_value = max(min(eval_value, 10.0), -10.0)  # Clip
        eval_value /= 10.0  # Normalize to [-1, 1]

        return board_tensor, torch.tensor(eval_value, dtype=torch.float32)
    
from torch.utils.data import DataLoader
import pandas as pd


dataset = pd.read_csv("val2.csv", encoding="utf-8")

dataset = EvalOnlyChessDataset(dataset)

#BATCH_SIZE = 512
#train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
#val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

In [24]:
@torch.no_grad()
def evaluate_board(board: chess.Board):
    """Fast board evaluation using neural network"""
    input_tensor = board_to_tensor(board)
    return model(input_tensor).item()

def order_moves(board, moves, maximize):
    """Order moves for better alpha-beta pruning"""
    move_scores = []
    
    for move in moves:
        score = 0
        
        # Prioritize captures
        if board.is_capture(move):
            captured_piece = board.piece_at(move.to_square)
            if captured_piece:
                # MVV-LVA: Most Valuable Victim - Least Valuable Attacker
                victim_value = [0, 1, 3, 3, 5, 9, 0][captured_piece.piece_type]
                attacker_value = [0, 1, 3, 3, 5, 9, 0][board.piece_at(move.from_square).piece_type]
                score += 1000 + victim_value * 10 - attacker_value
        
        # Prioritize checks
        board.push(move)
        if board.is_check():
            score += 500
        board.pop()
        
        # Prioritize promotions
        if move.promotion:
            score += 800
        
        # Prioritize center control
        if move.to_square in [chess.E4, chess.E5, chess.D4, chess.D5]:
            score += 50
        
        move_scores.append((score, move))
    
    # Sort moves by score (highest first for maximizing, lowest first for minimizing)
    move_scores.sort(key=lambda x: x[0], reverse=maximize)
    return [move for _, move in move_scores]

In [25]:
def minimax(board, depth, alpha, beta, maximizing):
    """Optimized minimax with alpha-beta pruning"""
    # Check transposition table
    board_key = board.fen()
    tt_value = transposition_table.get(board_key, depth)
    if tt_value is not None:
        return tt_value
    
    # Terminal conditions
    if depth == 0 or board.is_game_over():
        value = evaluate_board(board)
        transposition_table.store(board_key, depth, value)
        return value
    
    # Get and order legal moves
    legal_moves = list(board.legal_moves)
    if not legal_moves:
        value = evaluate_board(board)
        transposition_table.store(board_key, depth, value)
        return value
    
    # Order moves for better pruning
    ordered_moves = order_moves(board, legal_moves, maximizing)
    
    if maximizing:
        max_eval = -float('inf')
        for move in ordered_moves:
            board.push(move)
            eval_score = minimax(board, depth - 1, alpha, beta, False)
            board.pop()
            
            max_eval = max(max_eval, eval_score)
            alpha = max(alpha, eval_score)
            
            if beta <= alpha:
                break  # Alpha-beta cutoff
        
        transposition_table.store(board_key, depth, max_eval)
        return max_eval
    else:
        min_eval = float('inf')
        for move in ordered_moves:
            board.push(move)
            eval_score = minimax(board, depth - 1, alpha, beta, True)
            board.pop()
            
            min_eval = min(min_eval, eval_score)
            beta = min(beta, eval_score)
            
            if beta <= alpha:
                break  # Alpha-beta cutoff
        
        transposition_table.store(board_key, depth, min_eval)
        return min_eval

In [26]:
def find_best_move(fen, depth=4):
    """Find the best move using minimax with optimizations"""
    board = chess.Board(fen)
    best_move = None
    maximizing_player = board.turn == chess.WHITE
    best_score = -float('inf') if maximizing_player else float('inf')
    
    legal_moves = list(board.legal_moves)
    if not legal_moves:
        return None
    
    # Order moves for root search
    ordered_moves = order_moves(board, legal_moves, maximizing_player)
    
    print(f"Searching {len(legal_moves)} moves at depth {depth}...")
    
    for i, move in enumerate(ordered_moves):
        board.push(move)
        score = minimax(board, depth - 1, -float('inf'), float('inf'), not maximizing_player)
        board.pop()
        
        print(f"Move {i+1}/{len(ordered_moves)}: {move.uci()} | Eval: {score:.3f}")
        
        if maximizing_player and score > best_score:
            best_score = score
            best_move = move
        elif not maximizing_player and score < best_score:
            best_score = score
            best_move = move
    
    print(f"\nBest move: {best_move.uci()} | Eval: {best_score:.3f}")
    print(f"Transposition table size: {len(transposition_table.table)}")
    return best_move

def find_best_move_iterative(fen, max_depth=6, time_limit=None):
    """Find best move using iterative deepening"""
    import time
    start_time = time.time()
    
    board = chess.Board(fen)
    best_move = None
    
    for depth in range(1, max_depth + 1):
        if time_limit and time.time() - start_time > time_limit:
            break
            
        print(f"\n=== Depth {depth} ===")
        move = find_best_move(fen, depth)
        if move:
            best_move = move
    
    return best_move

In [29]:
if __name__ == "__main__":
    # Test position
    fen = "4rk2/p1p2ppp/1bQ5/3p4/3P4/2Pq4/P7/1KR5 w - - 0 1"
    
    # Quick search - DOBRO ZA BRZO RACUNANJE
    print("=== Quick Search ===")
    best_move = find_best_move(fen, depth=4)
    print("Best move:", best_move)
    
    # Iterative deepening search - DOBRO ZA DUBOKE POTEZE
    #print("\n=== Iterative Deepening Search ===")
    #best_move_iterative = find_best_move_iterative(fen, max_depth=4, time_limit=1000)
    #print("Best move (iterative):", best_move_iterative)
# 1m 21.8s

=== Quick Search (Depth 4) ===
Searching 3 moves at depth 5...
Move 1/3: b1b2 | Eval: -0.596
Move 2/3: b1a1 | Eval: -0.444
Move 3/3: c1c2 | Eval: -0.669

Best move: b1a1 | Eval: -0.444
Transposition table size: 40278
Best move: b1a1
